# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.linear_model import LogisticRegression
from gensim.parsing.preprocessing import STOPWORDS
import lightgbm as lgb
from catboost import CatBoostClassifier
import tensorflow_hub as hub
import tensorflow as tf
%matplotlib inline

# Se leen los .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test = pd.read_csv('csv/test.csv')

In [5]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


# Elaboración del modelo

Para este modelo se aplicará NLP para el procesamiento de los tweets y se utilizarán distintos árboles de decisión para realizar las predicciones:

### Preparación del set de datos

In [6]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflow-hub

In [7]:
def clean_text(df):
    
    tweets = df['text'].str.split()
    clean_tweets = []

    for tweet in tweets:
        clean_tweet = []
        for word in tweet:
            clean_word = re.sub('[^a-zA-Z]','', word)
            if(clean_word != '')&(clean_word not in STOPWORDS):
                clean_tweet.append(clean_word.lower())
        clean_tweets.append(" ".join(clean_tweet))

    df['clean_text'] = clean_tweets
    #df.set_index('id', inplace=True)
    df.drop(columns=['keyword','location','text'], inplace=True)
    
    return df

In [8]:
train = clean_text(train)
train.head()

,id,target,clean_text
0,1,1,our deeds reason earthquake may allah forgive
1,4,1,forest near la ronge sask canada
2,5,1,all residents asked shelter place notified off...
3,6,1,people receive wildfires evacuation orders cal...
4,7,1,just got sent photo ruby alaska smoke wildfire...


In [9]:
test = clean_text(test)
test.head()

,id,clean_text
0,0,just happened terrible car crash
1,2,heard earthquake different cities stay safe
2,3,forest spot pond geese fleeing street i save
3,9,apocalypse lighting spokane wildfires
4,11,typhoon soudelor kills china taiwan


### Embedding de los tweets con ELMo

In [10]:
# tf.compat.v1.disable_eager_execution()
# elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [11]:
# def elmo_vectoring(tweets):
    
#     embeddings = elmo(tweets.tolist(), signature="default", as_dict=True)["elmo"]
    
#     tf_session = tf.compat.v1.Session()
#     tf_session.run(tf.compat.v1.global_variables_initializer())
#     tf_session.run(tf.compat.v1.tables_initializer())
    
#     return tf_session.run(tf.reduce_mean(embeddings,1))

In [12]:
# list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
# list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [13]:
# elmo_train = [elmo_vectoring(x['clean_text']) for x in list_train]
# elmo_test = [elmo_vectoring(x['clean_text']) for x in list_test]

In [14]:
# elmo_train_new = np.concatenate(elmo_train, axis = 0)
# elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [15]:
# train_embedding_file = open("train_embedding_file.txt", "w")

# for i in elmo_train_new:
#     np.savetxt(train_embedding_file, i)

# train_embedding_file.close()

In [16]:
# test_embedding_file = open("test_embedding_file.txt", "w")

# for i in elmo_test_new:
#     np.savetxt(test_embedding_file, i)

# test_embedding_file.close()

In [17]:
elmo_train = np.loadtxt("train_embedding_file.txt").reshape(len(train.index),1024)
elmo_train.shape

(7613, 1024)

In [18]:
elmo_test = np.loadtxt("test_embedding_file.txt").reshape(len(test.index),1024)
elmo_test.shape

(3263, 1024)

### Se entrena el modelo

In [19]:
X, y = elmo_train, train.iloc[:,1].to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [20]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 20)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds)))
print('Precision Score : ' + str(precision_score(y_test,preds)))
print('Recall Score : ' + str(recall_score(y_test,preds)))
print('F1 Score : ' + str(f1_score(y_test,preds)))

/home/gonzams/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/gonzams/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score : 0.8031496062992126
Precision Score : 0.8268551236749117
Recall Score : 0.6985074626865672
F1 Score : 0.7572815533980582


In [21]:
df = pd.DataFrame(xg_reg.predict(X),columns=["elmo_score"])
df.head()

,elmo_score
0,0
1,1
2,1
3,1
4,1


In [22]:
final = df.to_csv('csv/solo_embedding_elmo_train.csv')

### Predicciones

In [23]:
test['target'] = xg_reg.predict(elmo_test)

In [24]:
test.drop(columns=['clean_text'], inplace=True)
test.set_index('id', inplace=True)
test.head()

,target
id,
0,0
2,0
3,1
9,1
11,1


In [25]:
final = test.to_csv('csv/submission_elmo.csv')

In [26]:
df = pd.DataFrame(xg_reg.predict(elmo_test),columns=["elmo_score"])
df.head()
final = df.to_csv('csv/solo_embedding_elmo_test.csv')